In [2]:
def eventsMatrix(design_file, task):
    """
    Returns the events matrix for the GLM. Works for the Sac/Pur Localisers and Sac/Pur Visual/Endogenous Localisers
    Parameters
    ----------
    design_file         : path to the tsv/csv file containing the events
    task                : task name (SacLoc / PurLoc / SacVELoc / PurVELoc)
    Returns
    -------
    new_events_glm - pandas DataFrame containing the events for the GLM 
    """
    import pandas as pd
    
    tr_dur = 1.2
    events = pd.read_table(design_file)

    if 'VE' not in task: # Sac/Pur Loc
        events_glm = events[['onset','duration','trial_type']].copy(deep=True)
        events_glm.replace({'trial_type': {3: 'Fix', 1: 'Sac', 2: 'Pur'}},inplace=True)
        events_glm.at[:,'duration'] = tr_dur
        events_glm.at[:,'onset'] = 0
        events_glm_groups = events_glm.groupby((events_glm.trial_type!=events_glm.trial_type.shift()).cumsum())

        new_events_glm = pd.DataFrame([], columns=['onset', 'duration', 'trial_type'])
        for idx, group in enumerate(events_glm_groups):
            onset = group[1]['onset'][group[1].index[0]]
            dur = sum(group[1]['duration'])
            ttype = group[1]['trial_type'][group[1].index[0]]

            new_events_glm = new_events_glm.append(pd.Series([onset,dur,ttype],index =['onset', 'duration', 'trial_type']), ignore_index=True)


    elif 'SacVE' in task: # Sac/Pur Loc: # Visual-Endogenous Sac/Pur Loc 
        events_glm = events[['onset','duration','trial_type', 'eyemov_vis_end']].copy(deep=True)
        events_glm.replace({'trial_type': {3: 'Fix', 1: 'Sac', 2: 'Pur'}},inplace=True)
        events_glm.replace({'eyemov_vis_end': {3: 'Fix', 1: 'SacExo', 2: 'SacEndo'}},inplace=True)
        events_glm.at[:,'duration'] = tr_dur
        events_glm.at[:,'onset'] = 0
        events_glm_groups = events_glm.groupby((events_glm.eyemov_vis_end!=events_glm.eyemov_vis_end.shift()).cumsum())

        new_events_glm = pd.DataFrame([], columns=['onset', 'duration', 'trial_type'])
        for idx, group in enumerate(events_glm_groups):
            onset = group[1]['onset'][group[1].index[0]]
            dur = sum(group[1]['duration'])
            vis_end = group[1]['eyemov_vis_end'][group[1].index[0]]

            new_events_glm = new_events_glm.append(pd.Series([onset,dur,vis_end],index =['onset', 'duration', 'trial_type']), ignore_index=True)
    
    elif 'PurVE' in task: # Sac/Pur Loc: # Visual-Endogenous Sac/Pur Loc 
        events_glm = events[['onset','duration','trial_type', 'eyemov_vis_end']].copy(deep=True)
        events_glm.replace({'trial_type': {3: 'Fix', 1: 'Sac', 2: 'Pur'}},inplace=True)
        events_glm.replace({'eyemov_vis_end': {3: 'Fix', 1: 'PurExo', 2: 'PurEndo'}},inplace=True)
        events_glm.at[:,'duration'] = tr_dur
        events_glm.at[:,'onset'] = 0
        events_glm_groups = events_glm.groupby((events_glm.eyemov_vis_end!=events_glm.eyemov_vis_end.shift()).cumsum())

        new_events_glm = pd.DataFrame([], columns=['onset', 'duration', 'trial_type', 'vis_end'])
        for idx, group in enumerate(events_glm_groups):
            onset = group[1]['onset'][group[1].index[0]]
            dur = sum(group[1]['duration'])
            ttype = group[1]['trial_type'][group[1].index[0]]
            vis_end = group[1]['eyemov_vis_end'][group[1].index[0]]

            new_events_glm = new_events_glm.append(pd.Series([onset,dur,vis_end],index =['onset', 'duration', 'trial_type']), ignore_index=True)
            
    elif 'pMF' in task:
        events_glm = events[['onset','duration','trial_type']].copy(deep=True)
        events_glm.replace({'trial_type': {3: 'Fix', 1: 'OM', 2: 'OM'}},inplace=True)
        events_glm.at[:,'duration'] = tr_dur
        events_glm.at[:,'onset'] = 0
        events_glm_groups = events_glm.groupby((events_glm.trial_type!=events_glm.trial_type.shift()).cumsum())
        
        new_events_glm = pd.DataFrame([], columns=['onset', 'duration', 'trial_type'])
        for idx, group in enumerate(events_glm_groups):
            onset = group[1]['onset'][group[1].index[0]]
            dur = sum(group[1]['duration'])
            ttype = group[1]['trial_type'][group[1].index[0]]

            new_events_glm = new_events_glm.append(pd.Series([onset,dur,ttype],index =['onset', 'duration', 'trial_type']), ignore_index=True)


    for idx in new_events_glm.index:
        if idx==0:
            new_events_glm.at[idx, 'onset'] = int(0)
        else:
            new_events_glm.at[idx, 'onset'] = new_events_glm.at[idx-1, 'onset'] + new_events_glm.at[idx-1, 'duration']

    return new_events_glm


In [13]:
# create design table
base_dir = '~/disks/meso_S/data/PredictEye'
subject = 'sub-01'
session = 'ses-01'
task = 'pMF'
design_file_run1 = '{base_dir}/bids_data/{subject}/{session}/func/{subject}_{session}_task-{task}_{run}_events.tsv'.\
                    format(base_dir=base_dir, subject=subject, session=session, task=task, run='run-01')


In [14]:
design_file_run1

'disks/meso_S/data/PredictEye/bids_data/sub-01/ses-01/func/sub-01_ses-01_task-pMF_run-01_events.tsv'

In [16]:
events_glm = eventsMatrix(design_file_run1, task)

FileNotFoundError: [Errno 2] No such file or directory: 'disks/meso_S/data/PredictEye/bids_data/sub-01/ses-01/func/sub-01_ses-01_task-pMF_run-01_events.tsv'

In [17]:
import os

In [21]:
os.getcwd()

'/home/mszinte/disks/meso_H/projects/PredictEye/mri_analysis/functions'